# Melatih Model

Tujuan utama dari pembelajaran mesin adalah untuk melatih model prediktif yang dapat digunakan oleh aplikasi. Di Azure Machine Learning, Anda dapat menggunakan skrip untuk melatih model yang memanfaatkan kerangka kerja pembelajaran mesin umum seperti Scikit-Learn, Tensorflow, PyTorch, SparkML, dan lainnya. Anda dapat menjalankan skrip pelatihan ini sebagai eksperimen untuk melacak metrik dan output, yang mencakup model terlatih.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Membuat skrip pelatihan

Anda akan menggunakan skrip Python untuk melatih model pembelajaran mesin berdasarkan data diabates, jadi mari kita mulai dengan membuat folder untuk skrip dan file data.

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

Sekarang Anda siap membuat skrip pelatihan dan menyimpannya di folder.

> **Catatan**: Kode ini *membuat* skrip - tidak menjalankannya!

In [ ]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Menjalankan skrip pelatihan sebagai eksperimen

Sekarang Anda siap menjalankan skrip sebagai eksperimen. Perhatikan bahwa lingkungan default tidak menyertakan paket **scikit-learn**, jadi Anda perlu menambahkannya secara eksplisit ke konfigurasi. Lingkungan conda dibuat sesuai permintaan saat pertama kali eksperimen dijalankan, dan di-cache untuk proses berikutnya yang menggunakan konfigurasi yang sama; jadi putaran pertama akan memakan waktu sedikit lebih lama.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment run
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

Anda dapat mengambil metrik dan output dari objek **Run**.

In [ ]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## Mendaftarkan model yang terlatih

Perhatikan bahwa keluaran eksperimen menyertakan file model terlatih (**diabetes_model.pkl**). Anda dapat mendaftarkan model ini di ruang kerja Azure Machine Learning Anda, sehingga memungkinkan untuk melacak versi model dan mengambilnya nanti.

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Catatan**: Kode di atas menggunakan *properti* dan *tag* untuk mencatat informasi tambahan tentang model. Perbedaan utama antara properti dan tag adalah bahwa properti tidak dapat diubah setelah model didaftarkan, sedangkan tag untuk model terdaftar dapat dimodifikasi.

## Buat skrip pelatihan berparameter

Anda dapat meningkatkan fleksibilitas eksperimen pelatihan Anda dengan menambahkan parameter ke skrip Anda, memungkinkan Anda untuk mengulangi eksperimen pelatihan yang sama dengan pengaturan yang berbeda. Dalam hal ini, Anda akan menambahkan parameter untuk tingkat regularisasi yang digunakan oleh algoritme regresi logistik saat melatih model.

Sekali lagi, mari kita mulai dengan membuat folder untuk skrip berparameter dan data pelatihan.

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

Sekarang mari kita buat skrip dengan argumen untuk hyperparameter tingkat regularisasi. Argumen dibaca menggunakan objek **argparse.ArgumentParser** Python.

In [ ]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the diabetes dataset
print("Loading Data...")
# load the diabetes dataset
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Jalankan skrip dengan argumen

Anda menjalankan skrip sebagai eksperimen seperti yang Anda lakukan sebelumnya, menggunakan kembali lingkungan yang Anda buat; tetapi kali ini Anda harus memberikan parameter **--reg_rate** yang diharapkan skrip sebagai argumen.

In [ ]:
# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

Sekali lagi, kita bisa mendapatkan metrik dan output dari proses yang telah selesai.

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## Daftarkan model versi baru

Sekarang setelah Anda melatih model baru, Anda dapat mendaftarkannya sebagai versi baru di ruang kerja.

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Anda juga dapat melihat model terdaftar di ruang kerja Anda di halaman **Model** di [Azure Machine Learning studio](https://ml.azure.com).

Jika Anda telah selesai menjelajah, Anda dapat menutup buku catatan ini dan mematikan instans komputasi Anda.